<a href="https://colab.research.google.com/github/Shabbarsyed/Cash-Ratio-Analysis/blob/main/Sentiment_Analys.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'imdb-dataset-of-50k-movie-reviews:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F134715%2F320111%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240219%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240219T215644Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D8d8530c90d99037f864df6a7bfd8785c987645455370e167151f1a66695becdf4ec16d37c163098ff6e303372b2d812d351f010e88e4c22a2c614143992ac4fe58b6a851136fce22e93fb99f31ce6872af5c4702660879c0bb3d3e74059e126ee7ab96781a162a814f3d092917f5f4b81cc6be3185c79fe15688bd11bacf99c41848a0c38c0c60f2bf46ecc934821fd8c349ec0f406745d9e5da95adc66d105833aab3d75e6d1628362a95e79adc536aae8bdf45993be20490ee824c56ec712766b8de402f8c6459a095f9c26a319e2231c17c6b0892e1ddb66497b6d789fb7ef3a774708cb34c8d24ba15812aebde217f03621092c812b19d88a90a307caa57'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


# Importing Dependencies

In [ ]:
!pip install tensorflow
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from numpy import array
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.preprocessing.text import one_hot, Tokenizer
from keras.models import Sequential
from keras.layers import Activation, Dropout, Dense
from tensorflow.keras.layers import LSTM, SimpleRNN, Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences

# Loading Data

In [ ]:
data = pd.read_csv('IMDB Dataset.csv')

In [ ]:
data.head()

In [ ]:
data.shape

# Data Pre-Processing

In [ ]:
data.isnull().any()

In [ ]:
data['review'][4]

# *Removing '<--->' Tag*
### Using Regular Expression we remove (<--><-->) type tags

In [ ]:
 Tag = re.compile(r'<[^>]+>')

 def remove(text):
  return Tag.sub('', text)

In [ ]:
new='''Petter Mattei\'s "Love in the Time of Money" is a visually stunning film to watch. Mr. Mattei offers us a vivid portrait about human relations. This is a movie that seems to be telling us what money, power and success do to people in the different situations we encounter. <br /><br />This being a variation on the Arthur Schnitzler\'s play about the same theme, the director transfers the action to the present time New York where all these different characters meet and connect. Each one is connected in one way, or another to the next person, but no one seems to know the previous point of contact. Stylishly, the film has a sophisticated luxurious look. We are taken to see how these people live and the world they live in their own habitat.<br /><br />The only thing one gets out of all these souls in the picture is the different stages of loneliness each one inhabits. A big city is not exactly the best place in which human relations find sincere fulfillment, as one discerns is the case with most of the people we encounter.<br /><br />The acting is good under Mr. Mattei\'s direction. Steve Buscemi, Rosario Dawson, Carol Kane, Michael Imperioli, Adrian Grenier, and the rest of the talented cast, make these characters come alive.<br /><br />We wish Mr. Mattei good luck and await anxiously for his next work'''

remove(new)

# *Filtering important words*
### Removing unimportant characters, numbers, spaces... etc using Regular Expression

In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
def preprocess(sen):
     sentence = sen.lower()

     # Remove html tags
     sentence = remove(sentence)

     # Remove punctuations and numbers
     sentence = re.sub('[^a-zA-Z]', ' ', sentence)

     # Single character removal
     sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

     # Remove multiple spaces
     sentence = re.sub(r'\s+', ' ', sentence)

     # Remove Stopwords
     pattern = re.compile(r'\b(' + r'|'.join(stopwords.words('english')) + r')\b\s*')
     sentence = pattern.sub('', sentence)

     return sentence

In [ ]:
preprocess(new)

# Getting Trainable Data

In [ ]:
X = []
sentences = list(data['review'])
for sen in sentences:
    X.append(preprocess(sen))

In [ ]:
X[4]

In [ ]:
Y = data['sentiment']

Y = np.array(list(map(lambda x: 1 if x=="positive" else 0, Y)))

# Using Train Test Split

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20, random_state=42)

# Tokenizing Words into numerical sequences

In [ ]:
word_tokenizer = Tokenizer()
word_tokenizer.fit_on_texts(X_train)

X_train = word_tokenizer.texts_to_sequences(X_train)
X_test = word_tokenizer.texts_to_sequences(X_test)

In [ ]:
vocab_length = len(word_tokenizer.word_index)+1
vocab_length

# Padding

In [ ]:
max_len=100
X_train=pad_sequences(X_train, padding='post',maxlen=max_len)
X_test=pad_sequences(X_test, padding='post',maxlen=max_len)

# Creating Embedding Matrix
### *using pre-trained GloVe Model (Download from GloVe Website)*

In [ ]:
from numpy import asarray
from numpy import zeros

embeddings_dictionary = dict()
glove_file = open('a2_glove.6B.100d.txt', encoding="utf8")

for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary [word] = vector_dimensions
glove_file.close()

In [ ]:
embedding_matrix = zeros((vocab_length, 100))
for word, index in word_tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector


embedding_matrix.shape

# RNN Model Building

In [ ]:
rnn_model = Sequential()
embedding_layer = Embedding(vocab_length, 100, weights=[embedding_matrix], input_length=max_len , trainable=False)

rnn_model.add(embedding_layer)
rnn_model.add(SimpleRNN(128))

rnn_model.add(Dense(1, activation='sigmoid'))

In [ ]:
rnn_model.summary()

In [ ]:
rnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

# Training RNN Model

In [ ]:
rnn_model_history = rnn_model.fit(X_train, Y_train, batch_size=128, epochs=10, verbose=1, validation_split=0.2)

# Evaluating RNN Model on test data

In [ ]:
rnn_score = rnn_model.evaluate(X_test, Y_test, verbose=1)
print("Test Accuracy:", rnn_score[1])

In [ ]:
plt.plot(rnn_model_history.history['acc'])
plt.plot(rnn_model_history.history['val_acc'])

plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train','test'], loc='upper left')
plt.show()

plt.plot(rnn_model_history.history['loss'])
plt.plot(rnn_model_history.history['val_loss'])

plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train','test'], loc='upper left')
plt.show()

# LSTM Model Building

In [ ]:
lstm_model = Sequential()
embedding_layer = Embedding(vocab_length, 100, weights=[embedding_matrix], input_length=max_len , trainable=False)

lstm_model.add(embedding_layer)
lstm_model.add(LSTM(128))

lstm_model.add(Dense(1, activation='sigmoid'))

In [ ]:
lstm_model.summary()

# Training LSTM Model

In [ ]:
lstm_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [ ]:
lstm_model_history = lstm_model.fit(X_train, Y_train, batch_size=128, epochs=10, verbose=1, validation_split=0.2)

# Evaluating LSTM Model on test Data

In [ ]:
lstm_score = lstm_model.evaluate(X_test, Y_test, verbose=1)
print("Test Accuracy:", lstm_score[1])

In [ ]:
plt.plot(lstm_model_history.history['acc'])
plt.plot(lstm_model_history.history['val_acc'])

plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train','test'], loc='upper left')
plt.show()

plt.plot(lstm_model_history.history['loss'])
plt.plot(lstm_model_history.history['val_loss'])

plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train','test'], loc='upper left')
plt.show()



*   RNN Model Gives around 55% accuracy
*   LSTM Model gives around 87% accuracy

LSTM models outperform simple RNNs due to their ability to capture long-term dependencies through memory cells and gating mechanisms, mitigating the vanishing gradient problem.
